In [1]:
from os import getcwd
from os.path import abspath, join

import numpy as np
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

from src.data.load_dataset import load_mnist
from src.models.networks import V1_mnist_RFNet, classical_RFNet
from src.models.utils import train, test

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

In [3]:
# load data
train_batch_size, train_percentage = 64, 0.01
train_loader, val_loader, test_loader = load_mnist(train_batch_size, train_percentage)

# training params
num_epochs = 10
step_size, gamma = 5, 0.1 # lr scheduler
num_trials = 50
log_interval = 100
num_neurons = sorted(set(np.logspace(0, 3.5, 50).astype('int')))
loss_fn = F.cross_entropy

### V1 RFNet with optimized parameters

In [ ]:
s, f, c = 5.34, 1.965, None
lr = 0.0031485838088746586

test_v1 = {'hidden_size': [], 'mean': [], 'std': []}
for hidden_size in tqdm(num_neurons):
    accuracy = []
    for trial in range(num_trials):
        model = V1_mnist_RFNet(hidden_size, s, f, c).to(device)
        optimizer = optim.Adam(model.parameters(), lr=lr)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

        for epoch in range(num_epochs):
            _ = train(log_interval, device, model, train_loader, optimizer, epoch, loss_fn, verbose=False)
            scheduler.step()
        accuracy.append(test(model, device, test_loader, verbose=False))
        
    test_v1['hidden_size'].append(hidden_size)
    test_v1['mean'].append(np.mean(accuracy))
    test_v1['std'].append(np.std(accuracy))

 11%|█▏        | 5/44 [16:36<2:26:56, 226.06s/it]

### Classical RFNet

In [ ]:
inp_size = (1, 28, 28)
lr = 0.01922083004518646

test_classical = {'hidden_size': [], 'mean': [], 'std': []} 
for hidden_size in tqdm(num_neurons):
    accuracy = []
    for trial in range(num_trials):
        model = classical_RFNet(inp_size, hidden_size).to(device)
        optimizer = optim.Adam(model.parameters(), lr=lr)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

        for epoch in range(num_epochs):
            _ = train(log_interval, device, model, train_loader, optimizer, epoch, loss_fn, verbose=False)
            scheduler.step()
        accuracy.append(test(model, device, test_loader, verbose=False))

    test_classical['hidden_size'].append(hidden_size)
    test_classical['mean'].append(np.mean(accuracy))
    test_classical['std'].append(np.std(accuracy))

### V1 RFNet with incompatible parameters $s=0.5$, $f=0.5$

In [ ]:
s, f, c = 0.5, 0.5, None
lr = 0.0031485838088746586

test_incompatible = {'hidden_size': [], 'mean': [], 'std': []}
for hidden_size in tqdm(num_neurons):
    accuracy = []
    for trial in range(num_trials):
        model = V1_mnist_RFNet(hidden_size, s, f, c).to(device)
        optimizer = optim.Adam(model.parameters(), lr=lr)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

        for epoch in range(num_epochs):
            _ = train(log_interval, device, model, train_loader, optimizer, epoch, loss_fn, verbose=False)
            scheduler.step()
        accuracy.append(test(model, device, test_loader, verbose=False))
        
    test_incompatible['hidden_size'].append(hidden_size)
    test_incompatible['mean'].append(np.mean(accuracy))
    test_incompatible['std'].append(np.std(accuracy))

In [7]:
# save
test_classical = {'hidden_size': [], 'mean': [], 'std': []} 
test_incompatible = {'hidden_size': [], 'mean': [], 'std': []}
test = {'v1': test_v1, 'classical': test_classical, 'incompatible': test_incompatible}
data_dir = abspath(join(getcwd(), '../../'))
with open(data_dir + '/models/results/mnist_clf/mnist_clf_s=%0.2f_f=%0.2f_fewshot_torch.pickle' % (s, f), 'wb') as handle:
    pickle.dump(test, handle, protocol=pickle.HIGHEST_PROTOCOL) 

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
plt.plot(test_v1['hidden_size'], test_v1['mean'])
plt.plot(test_classical['hidden_size'], test_classical['mean'])
plt.plot(test_incompatible['hidden_size'], test_incompatible['mean'])
plt.xlim([0, 1000])

In [6]:
list(zip(test_classical['hidden_size'], test_v1['mean']))

NameError: name 'test_classical' is not defined